In [2317]:
import numpy as np

In [2318]:
search_space = {
    'car_1': {'car_2': 4, 'car_4': 4, 'car_5': 3,'car_3': 4},
    'car_2': {'car_1': 4, 'car_4': 5, 'base_station_1': 6},
    'car_3': {'car_1': 4, 'car_6': 3},
    'car_4': {'car_1': 4, 'car_2': 5, 'car_5': 4, 'car_7': 1},
    'car_5': {'car_1': 3, 'car_4': 4, 'car_6': 5, 'car_7': 2, 'car_8': 7, 'car_9': 4},
    'car_6': {'car_3': 3, 'car_5': 5, 'car_9': 4},
    'car_7': {'car_4': 1, 'car_5': 2, 'car_8': 3, 'base_station_2': 6},
    'car_8': {'car_5': 7, 'car_7': 3, 'car_9': 4},
    'car_9': {'car_5': 4, 'car_6': 5, 'car_8': 4},
    'base_station_1': {'car_2': 6},
    'base_station_2': {'car_7': 6}
}

In [2319]:
known_costs = {}

In [2320]:
routes = {car: [] for car in search_space.keys()}

In [2321]:
visited_nodes = []

In [2322]:
unvisited_nodes = []

In [2323]:
def init_known_costs(init_node):
    global unvisited_nodes
    for car, bandwidth in search_space.items():
        if car != init_node:
            known_costs[car] = -np.inf
        else:
            known_costs[car] = 0
            routes[init_node] = [car]
    visited_nodes.append(init_node)
    unvisited_nodes = [node for node in search_space.keys() if node != init_node]

In [2324]:
def select_max_path(node):
    # get the minimum value in a node and set it as the max value
    max_val = min(search_space[node].values())
    # get the corresponding nodes (for the bandwidth set below)
    next_nodes = []
    print("------------For debugging----------------")
    print(search_space[node].items())
    for car, bandwidth in search_space[node].items():
        if car not in visited_nodes:
            # Check if this bandwidth is the new maximum
            if bandwidth > max_val:
                max_val = bandwidth
                # Update next_nodes with only the current highest cost
                next_nodes = [car]
            elif bandwidth == max_val:
                # If there is more than one node with that max value in the sub dict
                next_nodes.append(car)

        # Calculate the cummulative bandwidth to reach this neighbor
        new_cost = known_costs[node] + bandwidth

        if new_cost > known_costs[car]: ## there is a problem here
            routes[car] = routes[node] + [car]
            known_costs[car] = new_cost

    if len(next_nodes) == 0 and len(unvisited_nodes) > 0: ## review this part
        print("Starting search from root again")
        max_cost = max([known_costs[node] for node in unvisited_nodes])
        next_nodes = [node for node in unvisited_nodes if known_costs[node] == max_cost]
    print("-------------------------------------")
    # Mark the selected node as visited and remove it from unvisited nodes
    visited_nodes.append(next_nodes[0])
    unvisited_nodes.remove(next_nodes[0])

    # select the value in the array. If they are 2, select the first. But it honestly doesnt matter which you pick
    return next_nodes[0] 

### Dijkstra Algorithm Implementation

In [2326]:
def dijkstra_algorithm(init_node):
    init_known_costs(init_node)
    current_node = init_node
    i = 1
    while unvisited_nodes:
        print("********************************************************Iter "+str(i)+"***********************************************************")
        next_node = select_max_path(current_node)
        print("Next best node:"+next_node)
        print("Unvisited Nodes:"+ str(unvisited_nodes))
        print("Visited Nodes:"+ str(visited_nodes))
        print("Known Longest Routes:"+ str(routes))
        print("Known Largest Bandwidths:"+ str(known_costs))
        current_node = next_node
        i += 1

In [2327]:
dijkstra_algorithm('car_9')

********************************************************Iter 1***********************************************************
------------For debugging----------------
dict_items([('car_5', 4), ('car_6', 5), ('car_8', 4)])
-------------------------------------
Next best node:car_6
Unvisited Nodes:['car_1', 'car_2', 'car_3', 'car_4', 'car_5', 'car_7', 'car_8', 'base_station_1', 'base_station_2']
Visited Nodes:['car_9', 'car_6']
Known Longest Routes:{'car_1': [], 'car_2': [], 'car_3': [], 'car_4': [], 'car_5': ['car_9', 'car_5'], 'car_6': ['car_9', 'car_6'], 'car_7': [], 'car_8': ['car_9', 'car_8'], 'car_9': ['car_9'], 'base_station_1': [], 'base_station_2': []}
Known Largest Bandwidths:{'car_1': -inf, 'car_2': -inf, 'car_3': -inf, 'car_4': -inf, 'car_5': 4, 'car_6': 5, 'car_7': -inf, 'car_8': 4, 'car_9': 0, 'base_station_1': -inf, 'base_station_2': -inf}
********************************************************Iter 2***********************************************************
------------For 

In [2328]:
def get_optimal_route(destination):
    optimal_route = ''
    costs = []
    for idx, path in enumerate(routes[destination]):
        if idx > 0:
            costs.append(search_space[path][routes[destination][idx - 1]])
        if path != destination:
            optimal_route += path + " -> "
        else:
            optimal_route += path
    print("Optimal route to "+destination+": "+optimal_route)
    print("Data Transmission Rate: "+str(min(costs))+" Mbps")

In [2329]:
get_optimal_route('base_station_2')

Optimal route to base_station_2: car_9 -> car_6 -> car_5 -> car_8 -> car_7 -> base_station_2
Data Transmission Rate: 3 Mbps


In [2330]:
get_optimal_route('base_station_1')

Optimal route to base_station_1: car_9 -> car_6 -> car_5 -> car_8 -> car_7 -> car_4 -> car_2 -> base_station_1
Data Transmission Rate: 1 Mbps
